In [77]:
import numpy as np
from scipy.stats import distributions
normal = distributions.norm.pdf
import math

In [2]:
def getRows(dataBase,startInd,endInd):
    import sqlite3
    print(type(dataBase))
    input1 = dataBase+'.rdb'
    conn = sqlite3.connect(input1)
    c = conn.cursor()
    input2 = 'SELECT * FROM '+dataBase+'_ob'
    aa = c.execute(input2)
    rows = []
    count = 0
    for row in aa:
        if (count >= startInd) & (count < endInd):
            rows.append(row)
        count += 1
        if count == endInd:
            break
    return rows

In [3]:
def getStartTime(row):
    import datetime as dt
    start = dt.datetime(int(row[5][0:4]), int(row[5][5:7]), int(row[5][8:10]), int(row[5][11:13]), int(row[5][14:16]), int(float(row[5][17:-1])))
    return start

In [4]:
def getTimes(rows,startTime):
    import matplotlib.pyplot as plt
    import datetime as dt
    timeSec = [0]
    for afterInd in range(1,len(rows)):
        current = dt.datetime(int(rows[afterInd][5][0:4]), int(rows[afterInd][5][5:7]), int(rows[afterInd][5][8:10]), int(rows[afterInd][5][11:13]), int(rows[afterInd][5][14:16]), int(float(rows[afterInd][5][17:-1])))
        currentDiff = current - startTime
        currentDiffSeconds = currentDiff.days*(24*3600) + currentDiff.seconds
        timeSec.append(currentDiffSeconds)
    return timeSec

In [5]:
def extractTickData(rows,timeSec):
    import numpy as np
    buyPrices = []
    sellPrices = []
    buyTimes = []
    sellTimes = []
    buyQuantity = []
    buyLevels = []
    sellQuantity = []
    sellLevels = []
    for ii in range(len(rows)):
        if rows[ii][1] == 'buy':
            buyPrices.append(rows[ii][2])
            buyTimes.append(timeSec[ii])
            buyQuantity.append(rows[ii][3])
            buyLevels.append(rows[ii][4])
        else:
            sellPrices.append(rows[ii][2])
            sellTimes.append(timeSec[ii])
            sellQuantity.append(rows[ii][3])
            sellLevels.append(rows[ii][4])
    buyPrices = np.array(buyPrices)
    sellPrices = np.array(sellPrices)
    buyTimes = np.array(buyTimes)
    sellTimes = np.array(sellTimes)
    buyQuantity = np.array(buyQuantity)
    buyLevels = np.array(buyLevels)
    sellQuantity = np.array(sellQuantity)
    sellLevels = np.array(sellLevels)
    
    return buyPrices, sellPrices, buyTimes, sellTimes, buyQuantity, buyLevels, sellQuantity, sellLevels

In [6]:
def DHMS(seconds):
    import numpy as np
    days = np.floor(seconds/3600)
    hours = np.floor((seconds/3600) - days*24)
    minutes = np.floor(((seconds/3600) - days*24 - hours)*60)
    seconds = np.floor((((seconds/3600) - days*24 - hours)*60 - minutes)*60)
    return np.array([days,hours,minutes,seconds])

In [7]:
def plotTimes(sellTimes1,sellPrices1,buyTimes1,buyPrices1,sellTimes2,sellPrices2,buyTimes2,buyPrices2,label1,label2,start):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    import datetime as dt
    ax.plot(buyTimes1,buyPrices1,'g-',label=label1+' Buy Prices')
    ax.plot(sellTimes1,sellPrices1,'r-',label=label1+' Sell Prices')
    ax.plot(buyTimes2,buyPrices2,'b-',label=label2+' Buy Prices')
    ax.plot(sellTimes2,sellPrices2,'m-',label=label2+' Sell Prices')
    plt.show()
    labels = []
    for ii in range(0,len(ax.get_xticklabels())):
        if ax.get_xticklabels()[ii].get_text() != '':
            timeInc = int(ax.get_xticklabels()[ii].get_text())
            dhms = DHMS(timeInc)
            step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
            currentLabel = start + step
            labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
        else:
            labels.append(ax.get_xticklabels()[ii].get_text())
    fig, ax = plt.subplots()
    ax.plot(buyTimes1,buyPrices1,'g-',label=label1+' Buy Prices')
    ax.plot(sellTimes1,sellPrices1,'r-',label=label1+' Sell Prices')
    ax.plot(buyTimes2,buyPrices2,'b-',label=label2+' Buy Prices')
    ax.plot(sellTimes2,sellPrices2,'m-',label=label2+' Sell Prices')
    ax.set_xticklabels(labels)
    plt.xlabel('Time')
    plt.ylabel('Bitcoin Buy and Sell Prices')
    plt.title(label1+' and '+label2+' Bitcoin Bid/Ask')
    plt.legend()
    plt.show()

In [8]:
def timeFilterData(data,times,cutoff1,cutoff2):
    filteredData = data[(cutoff1 < times) & (times < cutoff2)]
    return filteredData

In [9]:
def trainData(sellTimes1, sellPrices1, buyTimes1, buyPrices1, sellTimes2, sellPrices2, buyTimes2, buyPrices2):
    import numpy as np
    from scipy import interpolate
    from sklearn import linear_model
    
    tck1 = interpolate.splrep(buyTimes1, buyPrices1, s=0)
    tck2 = interpolate.splrep(buyTimes2, buyPrices2, s=0)
    tck3 = interpolate.splrep(sellTimes2, sellPrices2, s=0)
    compare_buy_1 = interpolate.splev(sellTimes1, tck1, der=0)
    compare_buy_2 = interpolate.splev(sellTimes1, tck2, der=0)
    compare_sell_2 = interpolate.splev(sellTimes1, tck3, der=0)
    
    clfT = linear_model.LinearRegression()
    formatX = np.transpose(sellPrices1)
    formatY = np.transpose(compare_buy_2)
    newX = np.array([formatX]).T
    clfT.fit(newX, formatY)
    slopeToUse_1 = clfT.coef_
    interceptToUse_1 = clfT.intercept_

    clfT2 = linear_model.LinearRegression()
    formatX = np.transpose(compare_buy_1)
    formatY = np.transpose(compare_sell_2)
    newX = np.array([formatX]).T
    clfT2.fit(newX, formatY)
    slopeToUse_2 = clfT2.coef_
    interceptToUse_2 = clfT2.intercept_
    
    return slopeToUse_1, interceptToUse_1, slopeToUse_2, interceptToUse_2, compare_buy_1, compare_buy_2, compare_sell_2

In [10]:
def formatTradeData(sellTimes1,sellPrices1,buyTimes1,buyPrices1,sellTimes2,sellPrices2,buyTimes2,buyPrices2,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slope1,int1,slope2,int2):
    
    import numpy as np
    from scipy import interpolate
    from sklearn import linear_model
    from scipy.interpolate import interp1d
    """
    tck1 = interpolate.splrep(buyTimes1, buyPrices1, s=0)
    tck2 = interpolate.splrep(buyTimes2, buyPrices2, s=0)
    tck3 = interpolate.splrep(sellTimes2, sellPrices2, s=0)
    buy_1_sync = interpolate.splev(sellTimes1, tck1, der=0)
    buy_2_sync = interpolate.splev(sellTimes1, tck2, der=0)
    sell_2_sync = interpolate.splev(sellTimes1, tck3, der=0)
    """
    leftPoint = max([buyTimes1[0],buyTimes2[0],sellTimes2[0],sellTimes1[0]])
    rightPoint = min([buyTimes1[-1],buyTimes2[-1],sellTimes2[-1],sellTimes1[-1]])
    trade_timeline = sellTimes1[(sellTimes1 > leftPoint) & (sellTimes1 < rightPoint)]
    
    f1 = interp1d(buyTimes1, buyPrices1)
    f2 = interp1d(buyTimes2, buyPrices2)
    f3 = interp1d(sellTimes2, sellPrices2)
    f4 = interp1d(sellTimes1, sellPrices1)
    buy_1_sync = f1(trade_timeline)
    buy_2_sync = f2(trade_timeline)
    sell_2_sync = f3(trade_timeline)
    sell_1_sync = f4(trade_timeline)
    
    leftPoint_ref = max([buyTimes1_ref[0],buyTimes2_ref[0],sellTimes2_ref[0],sellTimes1_ref[0]])
    rightPoint_ref = min([buyTimes1_ref[-1],buyTimes2_ref[-1],sellTimes2_ref[-1],sellTimes1_ref[-1]])
    time_ref = sellTimes1_ref[(sellTimes1_ref > leftPoint_ref) & (sellTimes1_ref < rightPoint_ref)]
    
    """       
    tck1_ref = interpolate.splrep(buyTimes1_ref, buyPrices1_ref, s=0)
    tck2_ref = interpolate.splrep(buyTimes2_ref, buyPrices2_ref, s=0)
    tck3_ref = interpolate.splrep(sellTimes2_ref, sellPrices2_ref, s=0)
    buy_1_ref_sync = interpolate.splev(time_ref, tck1_ref, der=0)
    buy_2_ref_sync = interpolate.splev(time_ref, tck2_ref, der=0)
    sell_2_ref_sync = interpolate.splev(time_ref, tck3_ref, der=0)
    """
    g1 = interp1d(buyTimes1_ref, buyPrices1_ref)
    g2 = interp1d(buyTimes2_ref, buyPrices2_ref)
    g3 = interp1d(sellTimes2_ref, sellPrices2_ref)
    g4 = interp1d(sellTimes1_ref, sellPrices1_ref)
    
    buy_1_ref_sync = g1(time_ref)
    buy_2_ref_sync = g2(time_ref)
    sell_2_ref_sync = g3(time_ref)
    sell_1_ref_sync = g4(time_ref)
    
    zScore1 = np.zeros(len(trade_timeline)) 
    zScore2 = np.zeros(len(trade_timeline)) 
    
    import time
    startTime = time.time()
    time0 = trade_timeline[0]
    count = 0
    calcFlag = True
    for ii in range(0,len(zScore1)):
        if ii == 10**4:
            endTime = time.time()
            # print(endTime-startTime)
            # break
        currentTime = trade_timeline[ii]
        if (calcFlag == True):
            count = count + 1
            calcFlag = False
            sellPrices1_past = sell_1_ref_sync[time_ref < currentTime]
            buyPrices2_past = buy_2_ref_sync[time_ref < currentTime]
            time_past = time_ref[time_ref < currentTime]
           
            time_filt = time_past - time_past[-1]

            sellPrices1_past2 = sellPrices1_past[time_filt > -3600*backwardHours]
            buyPrices2_past2 = buyPrices2_past[time_filt > -3600*backwardHours]
            
            standardDev_1 = np.std((slope1*sellPrices1_past2 + int1) - buyPrices2_past2)
            meanHist_1 = np.mean((slope1*sellPrices1_past2 + int1) - buyPrices2_past2)

            buyPrices1_past = buy_1_ref_sync[time_ref < currentTime]
            sellPrices2_past = sell_2_ref_sync[time_ref < currentTime]

            buyPrices1_past2 = buyPrices1_past[time_filt > -3600*backwardHours]
            sellPrices2_past2 = sellPrices2_past[time_filt > -3600*backwardHours]

            standardDev_2 = np.std((slope2*buyPrices1_past2 + int2) - sellPrices2_past2)
            meanHist_2 = np.mean((slope2*buyPrices1_past2 + int2) - sellPrices2_past2)
        else:
            if (currentTime - time0 > 1*60*count):
                calcFlag = True
       
        zScore1[ii] = (((slope1*sell_1_sync[ii] + int1) - buy_2_sync[ii]) - meanHist_1)/standardDev_1
        zScore2[ii] = (((slope2*buy_1_sync[ii] + int2) - sell_2_sync[ii]) - meanHist_2)/standardDev_2
        """
        if (abs(zScore1[ii]) > 10) | (abs(zScore2[ii]) > 10):
            print(len(buyPrices1_past2))
            print(currentTime)
            print(time_past[-10:-1])
            print(currentTime - 3600*backwardHours)
        """
            
    return zScore1, zScore2, trade_timeline, sell_1_sync, buy_1_sync, buy_2_sync, sell_2_sync 

In [11]:
def pairs_trade(timeLine,sellPrices1,buyPrices1,sellPrices2,buyPrices2,zScore_1,zScore_2,trade_cutoff):
    import numpy as np
    bank1 = 10**4
    bank2 = 10**4
    coins1 = 0
    coins2 = 0

    bankTracker1 = [bank1]
    coinTracker1 = [coins1]
    naiveWorthTracker1 = [bank1 + coins1*buyPrices1[0]]

    bankTracker2 = [bank2]
    coinTracker2 = [coins2]
    naiveWorthTracker2 = [bank2 + coins2*buyPrices2[0]]

    zScoreTracker_1 = []
    zScoreTracker_2 = []
    cashOutTimes = []
    cashOut1 = []
    cashOut2 = []
    cashOutz1 = []
    cashOutz2 = []
    
    extremeFlag = 0
    betSize = 1
    
    for ii in range(0,len(timeLine)):
        
        currentTime = timeLine[ii]
        sellP1 = sellPrices1[ii]
        sellP2 = sellPrices2[ii]
        buyP1 = buyPrices1[ii]
        buyP2 = buyPrices2[ii]

        if zScore_1[ii] < -trade_cutoff:
            bank2 = bank2 + betSize*buyP2
            coins2 = coins2 - betSize
            bank1 = bank1 - betSize*sellP1
            coins1 = coins1 + betSize
            extremeFlag = 1
        if zScore_2[ii] > trade_cutoff:
            bank1 = bank1 + betSize*buyP1
            coins1 = coins1 - betSize
            bank2 = bank2 - betSize*sellP2
            coins2 = coins2 + betSize
            extremeFlag = 1
        if (abs(zScore_1[ii]) < .5) & (abs(zScore_2[ii]) < .5):
        
            if extremeFlag == 1:
                cashOutTimes.append(currentTime)
                if coins1 > 0:
                    bank1 = bank1 + coins1*buyP1
                    bank1 = bank1 - 0.002*coins1*buyP1
                else:
                    bank1 = bank1 + coins1*sellP1
                    bank1 = bank1 + 0.002*coins1*sellP1
                if coins2 > 0:
                    bank2 = bank2 + coins2*buyP2
                    bank2 = bank2 - 0.002*coins2*buyP2
                else:
                    bank2 = bank2 + coins2*sellP2 
                    bank2 = bank2 + 0.002*coins2*sellP2
                coins1 = 0
                coins2 = 0
                extremeFlag = 0
                cashOut1.append(bank1)
                cashOut2.append(bank2)
                cashOutz1.append(zScore_1[ii])
                cashOutz2.append(zScore_2[ii])
                
        bankTracker1.append(bank1)
        coinTracker1.append(coins1)
        naiveWorthTracker1.append(10**4 + 10*buyP1)

        bankTracker2.append(bank2)
        coinTracker2.append(coins2)
        naiveWorthTracker2.append(10**4 + 10*buyP2)

    bankTracker1 = np.array(bankTracker1)
    coinTracker1 = np.array(coinTracker1)
    naiveWorthTracker1 = np.array(naiveWorthTracker1)

    bankTracker2 = np.array(bankTracker2)
    coinTracker2 = np.array(coinTracker2)
    naiveWorthTracker2 = np.array(naiveWorthTracker2)
    cashOutTimes = np.array(cashOutTimes)
    cashOut1 = np.array(cashOut1)
    cashOut2 = np.array(cashOut2)
    cashOutz1 = np.array(cashOutz1)
    cashOutz2 = np.array(cashOutz2)
    
    return bankTracker1, naiveWorthTracker1, bankTracker2, naiveWorthTracker2, coinTracker1, coinTracker2, cashOutTimes, cashOut1, cashOut2, cashOutz1, cashOutz2

In [12]:
def plotReturn(timeLine,bankTracker1,naiveWorthTracker1,bankTracker2,naiveWorthTracker2,startTime,cashOutTimes,cashOutBank,label1,label2):
    import numpy as np
    fig, ax = plt.subplots()
    #ax.plot(timeLine,bankTracker1[1:],'g-',label=label1+' Portfolio Pairs Trading')
    #ax.plot(timeLine,10**4*np.ones(len(naiveWorthTracker1[1:])),'r-',label=u'Portfolio Inactive')
    #ax.plot(timeLine,bankTracker2[1:],'b-',label=label2+' Portfolio Pairs Trading')
    ax.plot(timeLine,bankTracker1[1:]+bankTracker2[1:],'c-',label=u'Total Portfolio Pairs Trading')
    ax.plot(timeLine,2*10**4*np.ones(len(naiveWorthTracker1[1:])),'y-',label=u'Total Inactive Trading')
    ax.plot(cashOutTimes,cashOutBank,'g*',label=u'Pairs Trading Cash Out')
    plt.show()
    labels = []
    for ii in range(0,len(ax.get_xticklabels())):
        if ax.get_xticklabels()[ii].get_text() != '':
            timeInc = int(ax.get_xticklabels()[ii].get_text())
            dhms = DHMS(timeInc)
            step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
            currentLabel = startTime + step
            labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
        else:
            labels.append(ax.get_xticklabels()[ii].get_text())
    fig, ax = plt.subplots()
    #ax.plot(timeLine,bankTracker1[1:],'g-',label=label1+' Portfolio Pairs Trading')
    #ax.plot(timeLine,10**4*np.ones(len(naiveWorthTracker1[1:])),'r-',label=u'Portfolio Inactive')
    #ax.plot(timeLine,bankTracker2[1:],'b-',label=label2+' Portfolio Pairs Trading')
    ax.plot(timeLine,bankTracker1[1:]+bankTracker2[1:],'c-',label=u'Total Portfolio Pairs Trading')
    ax.plot(timeLine,2*10**4*np.ones(len(naiveWorthTracker1[1:])),'y-',label=u'Total Inactive Trading')
    ax.plot(cashOutTimes,cashOutBank,'g*',label=u'Pairs Trading Cash Out')
    ax.set_xticklabels(labels)
    plt.xlabel('Time')
    plt.ylabel('Account Value')
    plt.title(label1+' and '+label2+' Pairs Trading')
    plt.legend(loc = 0)
    plt.show()

In [13]:
rows1 = getRows('huobi',20*10**6,24*10**6)

<class 'str'>


In [14]:
len(rows1)

4000000

In [15]:
rows2 = getRows('okcoin',0,4*10**6)

<class 'str'>


In [16]:
print(getStartTime(rows1[0]))

2013-12-31 15:25:23


In [17]:
print(getStartTime(rows2[0]))

2014-01-01 21:59:52


In [18]:
startTime = getStartTime(rows1[0])
times1 = getTimes(rows1,startTime)
times2 = getTimes(rows2,startTime)
buyPrices1, sellPrices1, buyTimes1, sellTimes1, buyQuantity1, buyLevels1, sellQuantity1, sellLevels1 = extractTickData(rows1,times1)
buyPrices2, sellPrices2, buyTimes2, sellTimes2, buyQuantity2, buyLevels2, sellQuantity2, sellLevels2 = extractTickData(rows2,times2)

In [19]:
plotTimes(sellTimes1,sellPrices1,buyTimes1,buyPrices1,sellTimes2,sellPrices2,buyTimes2,buyPrices2,'huobi','okcoin',startTime)

In [20]:
import datetime as dt
startCutoff = dt.datetime(2014,1,2,10,0,0) - startTime
startCutOffSeconds = startCutoff.days*(24*3600) + startCutoff.seconds
trainCutoff = dt.datetime(2014,1,2,23,0,0) - startTime
trainCutoffSeconds = trainCutoff.days*(24*3600) + trainCutoff.seconds
endCutoff = dt.datetime(2014,1,4,2,0,0) - startTime
endCutoffSeconds = endCutoff.days*(24*3600) + endCutoff.seconds

In [21]:
buyPrices1_train = timeFilterData(buyPrices1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellPrices1_train = timeFilterData(sellPrices1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
buyTimes1_train = timeFilterData(buyTimes1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellTimes1_train = timeFilterData(sellTimes1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
buyQuantity1_train = timeFilterData(buyQuantity1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
buyLevels1_train = timeFilterData(buyLevels1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellQuantity1_train = timeFilterData(sellQuantity1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
sellLevels1_train = timeFilterData(sellLevels1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)

buyPrices2_train = timeFilterData(buyPrices2,buyTimes2,startCutOffSeconds,trainCutoffSeconds)
sellPrices2_train = timeFilterData(sellPrices2,sellTimes2,startCutOffSeconds,trainCutoffSeconds)
buyTimes2_train = timeFilterData(buyTimes2,buyTimes2,startCutOffSeconds,trainCutoffSeconds)
sellTimes2_train = timeFilterData(sellTimes2,sellTimes2,startCutOffSeconds,trainCutoffSeconds)
buyQuantity2_train = timeFilterData(buyQuantity2,buyTimes2,startCutOffSeconds,trainCutoffSeconds)
buyLevels2_train = timeFilterData(buyLevels2,buyTimes2,startCutOffSeconds,trainCutoffSeconds)
sellQuantity2_train = timeFilterData(sellQuantity2,sellTimes2,startCutOffSeconds,trainCutoffSeconds)
sellLevels2_train = timeFilterData(sellLevels2,sellTimes2,startCutOffSeconds,trainCutoffSeconds)

buyPrices1_trade = timeFilterData(buyPrices1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellPrices1_trade = timeFilterData(sellPrices1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
buyTimes1_trade = timeFilterData(buyTimes1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellTimes1_trade = timeFilterData(sellTimes1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
buyQuantity1_trade = timeFilterData(buyQuantity1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
buyLevels1_trade = timeFilterData(buyLevels1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellQuantity1_trade = timeFilterData(sellQuantity1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
sellLevels1_trade = timeFilterData(sellLevels1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)

buyPrices2_trade = timeFilterData(buyPrices2,buyTimes2,trainCutoffSeconds,endCutoffSeconds)
sellPrices2_trade = timeFilterData(sellPrices2,sellTimes2,trainCutoffSeconds,endCutoffSeconds)
buyTimes2_trade = timeFilterData(buyTimes2,buyTimes2,trainCutoffSeconds,endCutoffSeconds)
sellTimes2_trade = timeFilterData(sellTimes2,sellTimes2,trainCutoffSeconds,endCutoffSeconds)
buyQuantity2_trade = timeFilterData(buyQuantity2,buyTimes2,trainCutoffSeconds,endCutoffSeconds)
buyLevels2_trade = timeFilterData(buyLevels2,buyTimes2,trainCutoffSeconds,endCutoffSeconds)
sellQuantity2_trade = timeFilterData(sellQuantity2,sellTimes2,trainCutoffSeconds,endCutoffSeconds)
sellLevels2_trade = timeFilterData(sellLevels2,sellTimes2,trainCutoffSeconds,endCutoffSeconds)

In [22]:
sellTimes1_train =  sellTimes1_train[sellLevels1_train == 1]
sellPrices1_train = sellPrices1_train[sellLevels1_train == 1]
buyTimes1_train = buyTimes1_train[buyLevels1_train == 1]
buyPrices1_train = buyPrices1_train[buyLevels1_train == 1]
sellTimes2_train = sellTimes2_train[sellLevels2_train == 1]
sellPrices2_train = sellPrices2_train[sellLevels2_train == 1]
buyTimes2_train = buyTimes2_train[buyLevels2_train == 1]
buyPrices2_train = buyPrices2_train[buyLevels2_train == 1]
slopeToUse_1, interceptToUse_1, slopeToUse_2, interceptToUse_2, buyPrices1_train_syn, buyPrices2_train_sync, sellPrices2_train_sync = trainData(sellTimes1_train, sellPrices1_train, buyTimes1_train, buyPrices1_train, sellTimes2_train, sellPrices2_train, buyTimes2_train, buyPrices2_train)

In [23]:
slopeToUse_1

array([ 0.92720093])

In [24]:
interceptToUse_1

317.69723545358011

In [25]:
slopeToUse_2

array([ 0.92795483])

In [26]:
interceptToUse_2

320.53174772890725

In [27]:
import matplotlib.pyplot as plt
plt.plot(sellPrices1_train,buyPrices2_train_sync,'b.')
plt.plot(sellPrices1_train,slopeToUse_1*sellPrices1_train + interceptToUse_1,'r')
plt.xlabel('Huobi sell prices')
plt.ylabel('Okcoin sell prices')
plt.title('Best Fit Parameters')
plt.show()

In [28]:
buyPrices1_trade = buyPrices1_trade[buyLevels1_trade == 1]
sellPrices1_trade = sellPrices1_trade[sellLevels1_trade == 1]
buyTimes1_trade = buyTimes1_trade[buyLevels1_trade == 1]
sellTimes1_trade = sellTimes1_trade[sellLevels1_trade == 1]

buyPrices1_ref = buyPrices1[buyLevels1 == 1]
sellPrices1_ref = sellPrices1[sellLevels1 == 1]
buyTimes1_ref = buyTimes1[buyLevels1 == 1]
sellTimes1_ref = sellTimes1[sellLevels1 == 1]

buyPrices2_trade = buyPrices2_trade[buyLevels2_trade == 1]
sellPrices2_trade = sellPrices2_trade[sellLevels2_trade == 1]
buyTimes2_trade = buyTimes2_trade[buyLevels2_trade == 1]
sellTimes2_trade = sellTimes2_trade[sellLevels2_trade == 1]

buyPrices2_ref = buyPrices2[buyLevels2 == 1]
sellPrices2_ref = sellPrices2[sellLevels2 == 1]
buyTimes2_ref = buyTimes2[buyLevels2 == 1]
sellTimes2_ref = sellTimes2[sellLevels2 == 1]

In [29]:
plt.plot(buyPrices2_trade)
plt.show()

In [30]:
def shallowCopy(array):              # Create shallow copies for arrays to prevent different variables from pointing to the same
    if len(array.shape) > 1:         # memory location
        newArray = np.zeros((array.shape[0],array.shape[1]))
        for ii in range(0,array.shape[0]):
            for jj in range(0,array.shape[1]):
                newArray[ii,jj] = array[ii,jj]
    else:
        newArray = np.zeros(len(array))
        for ii in range(0,len(array)):
            newArray[ii] = array[ii]
    return newArray

In [31]:
def kmeans(data,K):
    N = data.shape[1]
    D = data.shape[0]
    responsibilities = np.zeros((K,N))
    for ii in range(0,N):
        u = np.random.uniform(0,1)
        init = np.floor(u*K)
        responsibilities[init,ii] = 1
    oldResp = np.zeros((K,N))
    objectiveStream = []
    while sum(abs(responsibilities.flatten() - oldResp.flatten())) > 0:
        meansK = np.zeros((D,K))
        nK = np.zeros(K)
        for ii in range(0,K):
            nK[ii] = sum(responsibilities[ii,:])
            for jj in range(0,N):
                meansK[:,ii] = meansK[:,ii] + responsibilities[ii,jj]*data[:,jj]
            if nK[ii] > 0:
                meansK[:,ii] = meansK[:,ii]/nK[ii]
            else:
                u = np.random.uniform(0,1)
                pickInd = int(np.floor(u*N))
                meansK[:,ii] = data[:,pickInd]
        oldResp = shallowCopy(responsibilities)
        responsibilities = np.zeros((K,N))
        respVec = np.zeros(N)
        objectiveValue = 0
        objectiveVec = np.zeros(N)
        for ii in range(0,N):
            minAssignment = 0
            minDistance = np.sqrt(sum((data[:,ii]-meansK[:,0])**2))
            for jj in range(1,K):
                cDistance = np.sqrt(sum((data[:,ii]-meansK[:,jj])**2))
                if cDistance < minDistance:
                    minDistance = cDistance
                    minAssignment = jj
            responsibilities[minAssignment,ii] = 1
            respVec[ii] = minAssignment
            objectiveValue = objectiveValue + minDistance
            objectiveVec[ii] = minDistance
        objectiveStream.append(objectiveValue)
        # print("Objective Value is: %f" % objectiveValue)
    return responsibilities, respVec, meansK, objectiveValue, objectiveVec, np.array(objectiveStream)

In [32]:
def kmeansPlus(data,K):
    N = data.shape[1]
    D = data.shape[0]
    responsibilities = np.zeros((K,N))
    meansInit = np.zeros((D,K))
    u = np.random.uniform(0,1)
    randInd = np.floor(u*N)
    meansInit[:,0] = data[:,randInd]
    minDist = np.zeros(N)
    for kk in range(1,K):
        for ii in range(0,N):
            minDist[ii] = np.sqrt(sum((data[:,ii]-meansInit[:,0])**2))
            for jj in range(0,kk-1):
                currentDist = np.sqrt(sum((data[:,ii]-meansInit[:,jj])**2))
                if currentDist < minDist[ii]:
                    minDist[ii] = currentDist
        wDist = (minDist**2)/sum(minDist**2)
        resultArray = np.random.multinomial(1, list(wDist), size=1)
        for dd in range(0,N):
            if resultArray[0][dd] == 1:
                chosenInd = dd
        meansInit[:,kk] = data[:,chosenInd]
    for ii in range(0,N):
        minAssignment = 0
        minDistance = np.sqrt(sum((data[:,ii]-meansInit[:,0])**2))
        for jj in range(1,K):
            cDistance = np.sqrt(sum((data[:,ii]-meansInit[:,jj])**2))
            if cDistance < minDistance:
                minDistance = cDistance
                minAssignment = jj
        responsibilities[minAssignment,ii] = 1
            
    oldResp = np.zeros((K,N))
    objectiveStream = []
    while sum(abs(responsibilities.flatten() - oldResp.flatten())) > 0:
        meansK = np.zeros((D,K))
        nK = np.zeros(K)
        for ii in range(0,K):
            nK[ii] = sum(responsibilities[ii,:])
            for jj in range(0,N):
                meansK[:,ii] = meansK[:,ii] + responsibilities[ii,jj]*data[:,jj]
            if nK[ii] > 0:
                meansK[:,ii] = meansK[:,ii]/nK[ii]
            else:
                u = np.random.uniform(0,1)
                pickInd = int(np.floor(u*N))
                meansK[:,ii] = data[:,pickInd]
        oldResp = shallowCopy(responsibilities)
        responsibilities = np.zeros((K,N))
        respVec = np.zeros(N)
        objectiveValue = 0
        objectiveVec = np.zeros(N)
        for ii in range(0,N):
            minAssignment = 0
            minDistance = np.sqrt(sum((data[:,ii]-meansK[:,0])**2))
            for jj in range(1,K):
                cDistance = np.sqrt(sum((data[:,ii]-meansK[:,jj])**2))
                if cDistance < minDistance:
                    minDistance = cDistance
                    minAssignment = jj
            responsibilities[minAssignment,ii] = 1
            respVec[ii] = minAssignment
            objectiveValue = objectiveValue + minDistance
            objectiveVec[ii] = minDistance
        objectiveStream.append(objectiveValue)
        #print("Objective Value is: %f" % objectiveValue)
    return responsibilities, respVec, meansK, objectiveValue, objectiveVec, np.array(objectiveStream)

In [140]:
def getViterbiPath(prices,piVec,transitionMatrix,meanMatrix,varMatrix):
    nStates = transitionMatrix.shape[0]
    nTime = len(prices[0])
    deltaMat = np.zeros((nStates,nTime))
    psiMat = np.zeros((nStates,nTime-1))
    viterbiPath = np.zeros(nTime)
    for ii in range(0,nTime):
        if ii == 0:
            prob = normal(prices[0][ii],meanMatrix,np.sqrt(varMatrix))
            deltaMat[:,ii] = np.log(piVec*prob)
        else:
            for jj in range(0,nStates):
                deltaMat[jj,ii] = max(deltaMat[:,ii-1] + np.log(transitionMatrix[:,jj]))+ np.log(normal(prices[0][ii],meanMatrix[0,jj],np.sqrt(varMatrix[0,jj])))
                psiMat[jj,ii-1] = np.argmax(deltaMat[:,ii-1]+np.log(transitionMatrix[:,jj]))
        if all(deltaMat[:,ii] == -np.inf):
            print("All -inf")
    viterbiPath[-1] = np.argmax(deltaMat[:,nTime-1])
    for ii in range(2,nTime+1):
        viterbiPath[-ii] = psiMat[viterbiPath[-ii+1],nTime-ii]
    return viterbiPath

In [141]:
def getLogLik(prices,stateVec,piVec,transitionMatrix,meanMatrix,varMatrix):
    logLik = 0
    for ii in range(0,len(prices[0])):
        logLik = logLik + np.log(normal(prices[0][ii],meanMatrix[0,stateVec[0][ii]],np.sqrt(varMatrix[0,stateVec[0][ii]])))
        if ii == 0:
            logLik = logLik + np.log(piVec[0,stateVec[0][ii]])
        if ii < len(prices)-1:
            logLik = logLik + np.log(transitionMatrix[stateVec[0][ii],stateVec[0][ii+1]])
    return logLik

In [142]:
def trainDataHMM_0(prices, nStates, nComponents, logLikThresh):
    nTime = len(prices[0])
    
    meanNumbers = np.random.uniform(0,1,nComponents*nStates)
    varNumbers = np.random.uniform(0,1,nComponents*nStates)
    transitionNumbers = np.random.uniform(0,1,nStates*nStates)
    transitionMatrix = np.reshape(transitionNumbers,(nStates,nStates))
    for ii in range(0,nStates):
        transitionMatrix[ii,:] = np.copy(transitionMatrix[ii,:]/sum(transitionMatrix[ii,:]))
    piNumbers = np.random.uniform(0,1,nStates)
    piNumbers = np.copy(piNumbers/sum(piNumbers))
    piVec = np.reshape(piNumbers,(1,nStates))
    
    meanMatrix = np.reshape(meanNumbers,(nComponents,nStates))
    varMatrix = np.reshape(varNumbers,(nComponents,nStates))
    
    stateVec = getViterbiPath(prices,piVec,transitionMatrix, meanMatrix,varMatrix)
    stateVec = np.reshape(stateVec,(1,len(stateVec)))

    currentLogLik = getLogLik(prices,stateVec,piVec,transitionMatrix,meanMatrix,varMatrix)
    likVec = [currentLogLik]
    
    print("Initial Log Lik is: ", currentLogLik)
    print("Initial pi is: ", piVec)
    print("Initial transition is: ", transitionMatrix)
    print("Initial mean is: ", meanMatrix)
    print("Initial var is: ", varMatrix)
    print("Initial path is: ", stateVec)
    
    newLogLik = 10

    while abs(newLogLik - currentLogLik) > logLikThresh:
        viterbiPath = getViterbiPath(prices,piVec,transitionMatrix, meanMatrix,varMatrix)
        viterbiPath = np.reshape(viterbiPath,(1,len(viterbiPath)))
        intermediateLogLik = getLogLik(prices,viterbiPath,piVec,transitionMatrix,meanMatrix,varMatrix)
        print("Intermediate log lik is: ",intermediateLogLik)
        
        meanMatrixNew = np.zeros((nComponents,nStates))
        varMatrixNew = np.zeros((nComponents,nStates))
        for ii in range(0,nStates):
            nGroup = sum(sum(viterbiPath == ii))
            if nGroup == 0:
                print('empty cluster',ii)
            dataCurrent = prices[0,viterbiPath == ii]
            if nComponents > 1:
                respMatS, stateVec, meansKS, objValueS, objVecS, ojbStreamS = kmeansPlus(dataCurrent,nComponents)
                for jj in range(0,nComponents):
                    meanMatrixNew[ii,jj] = np.mean(dataCurrent[stateVec == jj])
                    varMatrixNew[ii,jj] = np.var(dataCurrent[stateVec == jj])   
            else:
                meanMatrixNew[0,ii] = np.mean(dataCurrent)
                varMatrixNew[0,ii] = np.var(dataCurrent)
     
        newLogLik = getLogLik(prices,viterbiPath,piVec,transitionMatrix,meanMatrixNew,varMatrixNew)
        
        if newLogLik > currentLogLik:
            likVec.append(newLogLik)
            meanMatrix = np.copy(meanMatrixNew)
            varMatrix = np.copy(varMatrixNew)
            stateVec = np.copy(viterbiPath)
            currentLogLik = newLogLik
            
            print("New Log Lik is: ", currentLogLik)
            print("New mean is: ", meanMatrix)
            print("New var is: ", varMatrix)
            print("New path is: ", stateVec)
        else:
            print("Worse New Log Lik is: ", newLogLik)
            print("Worse path is: ",viterbiPath)
            break
        
       
    return piVec, transitionMatrix, meanMatrix, varMatrix, likVec

In [143]:
def trainDataHMM(prices, nStates, nComponents, logLikThresh):
    nTime = len(prices[0])
    respMat, stateVec, meansK, objValue, objVec, ojbStream = kmeansPlus(prices,nStates)
    stateVec = np.reshape(stateVec,(1,len(stateVec)))
    meanMatrix = np.zeros((nComponents,nStates))
    varMatrix = np.zeros((nComponents,nStates))
    for ii in range(0,nStates):
        nGroup = sum(stateVec[0] == ii)
        if nGroup == 0:
            print('initial empty cluster',ii)
        dataCurrent = prices[0,stateVec[0] == ii]
        if nComponents > 1:
            respMatS, stateVecS, meansKS, objValueS, objVecS, ojbStreamS = kmeansPlus(dataCurrent,nComponents)
            for jj in range(0,nComponents):
                meanMatrix[ii,jj] = np.mean(dataCurrent[stateVecS[0] == jj])
                varMatrix[ii,jj] = np.var(dataCurrent[stateVecS[0] == jj])
        else:
            meanMatrix[0,ii] = np.mean(dataCurrent)
            varMatrix[0,ii] = np.var(dataCurrent)
            
    transitionMatrix = np.zeros((nStates,nStates))
    for ii in range(0,nStates):
        for jj in range(0,nStates):
            den = 0
            for kk in range(0,nTime-1):
                if (stateVec[0,kk] == ii) & (stateVec[0,kk+1] == jj):
                    transitionMatrix[ii,jj] = transitionMatrix[ii,jj] + 1
                if stateVec[0,kk] == ii:
                    den = den + 1
            if den > 0:
                transitionMatrix[ii,jj] = transitionMatrix[ii,jj]/den
    
    initialObs = prices[0][0]
    invDist = 1/abs(meansK-initialObs)
    piVec = invDist/sum(invDist[0])

    currentLogLik = getLogLik(prices,stateVec,piVec,transitionMatrix,meanMatrix,varMatrix)
    print("Initial Log Lik is: ", currentLogLik)
    print("Initial pi is: ", piVec)
    print("Initial transition is: ", transitionMatrix)
    print("Initial mean is: ", meanMatrix)
    print("Initial var is: ", varMatrix)
    print("Initial path is: ", stateVec)
    likVec = [currentLogLik]
    
    newLogLik = 10

    while abs(newLogLik - currentLogLik) > logLikThresh:
        viterbiPath = getViterbiPath(prices,piVec,transitionMatrix, meanMatrix,varMatrix)
        viterbiPath = np.reshape(viterbiPath,(1,len(viterbiPath)))
        intermediateLogLik = getLogLik(prices,viterbiPath,piVec,transitionMatrix,meanMatrix,varMatrix)
        print("Intermediate log lik is: ",intermediateLogLik)
        
        meanMatrixNew = np.zeros((nComponents,nStates))
        varMatrixNew = np.zeros((nComponents,nStates))
        for ii in range(0,nStates):
            nGroup = sum(viterbiPath[0] == ii)
            if nGroup == 0:
                print('empty cluster',ii)
            dataCurrent = prices[0,viterbiPath[0] == ii]
            if nComponents > 1:
                respMatS, stateVec, meansKS, objValueS, objVecS, ojbStreamS = kmeansPlus(dataCurrent,nComponents)
                for jj in range(0,nComponents):
                    meanMatrixNew[ii,jj] = np.mean(dataCurrent[stateVec[0] == jj])
                    varMatrixNew[ii,jj] = np.var(dataCurrent[stateVec[0] == jj])   
            else:
                meanMatrixNew[0,ii] = np.mean(dataCurrent)
                varMatrixNew[0,ii] = np.var(dataCurrent)
        transitionMatrixNew = np.zeros((nStates,nStates))
      
        for ii in range(0,nStates):
            for jj in range(0,nStates):
                den = 0
                for kk in range(0,nTime-1):
                    if (viterbiPath[0,kk] == ii) & (viterbiPath[0,kk+1] == jj):
                        transitionMatrixNew[ii,jj] = transitionMatrixNew[ii,jj] + 1
                    if viterbiPath[0,kk] == ii:
                        den = den + 1
                if den > 0:
                    transitionMatrixNew[ii,jj] = transitionMatrixNew[ii,jj]/den
        meansKNew = np.zeros((nComponents,nStates))
        for ii in range(0,nStates):
            meansKNew[0,ii] = np.mean(prices[0][viterbiPath[0] == ii])
        invDistNew = 1/abs(meansKNew-initialObs)
        piVecNew = invDistNew/sum(invDistNew[0])
     
        newLogLik = getLogLik(prices,viterbiPath,piVecNew,transitionMatrixNew,meanMatrixNew,varMatrixNew)
        
        if newLogLik > currentLogLik:
            likVec.append(newLogLik)
            piVec = np.copy(piVecNew)
            transitionMatrix = np.copy(transitionMatrixNew)
            meanMatrix = np.copy(meanMatrixNew)
            varMatrix = np.copy(varMatrixNew)
            stateVec = np.copy(viterbiPath)
            currentLogLik = newLogLik
            
            print("New Log Lik is: ", currentLogLik)
            print("New pi is: ", piVec)
            print("New transition is: ", transitionMatrix)
            print("New mean is: ", meanMatrix)
            print("New var is: ", varMatrix)
            print("New path is: ", stateVec)
        else:
            print("Worse Candidate Log Lik is: ", newLogLik)
            print("Worse path is: ",viterbiPath)
            break
        
    return piVec, transitionMatrix, meanMatrix, varMatrix, likVec

In [134]:
trainBuy = np.reshape(buyPrices2_trade[0:10**4],(1,10**4))
piVec, transitionMatrix, meanMatrix, varMatrix, likVec = trainDataHMM(trainBuy, 2, 1, 0.1)

Initial Log Lik is:  -47818.4456546
Initial pi is:  [[ 0.82717119  0.17282881]]
Initial transition is:  [[  9.98708567e-01   1.29143349e-03]
 [  9.34055670e-04   9.99065944e-01]]
Initial mean is:  [[ 4613.39978261  4773.92842548]]
Initial var is:  [[ 1064.24570901   674.40131144]]
Initial path is:  [[ 0.  0.  0. ...,  1.  1.  1.]]
[[ -5.43830125e+00  -1.06881512e+01  -1.59380011e+01 ...,  -4.77881852e+04
   -4.77925072e+04  -4.77968276e+04]
 [ -3.64620476e+01  -4.67968976e+01  -5.20467475e+01 ...,  -4.77710288e+04
   -4.77753492e+04  -4.77796696e+04]]
Intermediate log lik is:  -47748.3558349
New Log Lik is:  -47701.2432821
New pi is:  [[ 0.82290649  0.17709351]]
New transition is:  [[  9.99577435e-01   4.22564969e-04]
 [  1.89897455e-04   9.99810103e-01]]
New mean is:  [[ 4614.94332981  4775.19297513]]
New var is:  [[ 1172.0086501    587.04984714]]
New path is:  [[ 0.  0.  0. ...,  1.  1.  1.]]


In [193]:
def getAlpha(currentWindow,nStates,piVec,transitionMatrix,meanMatrix,varMatrix):
    
    alphaMat = np.zeros((nStates,len(currentWindow)))
    alphaMat[:,0] = piVec*normal(currentWindow[0],meanMatrix,np.sqrt(varMatrix))
    alphaMat[:,0] = np.copy(alphaMat[:,0]/sum(alphaMat[:,0]))
    if any(alphaMat[:,0] == 0):
        print('zero prob exist')
    alphaMatUnNorm = np.zeros((nStates,len(currentWindow)))
    alphaMatUnNorm[:,0] = piVec*normal(currentWindow[0],meanMatrix,np.sqrt(varMatrix))
   
    #logAlpha = np.zeros((nStates,len(currentWindow)))
    #logAlpha[:,0] = np.log(piVec*normal(currentWindow[0],meanMatrix,np.sqrt(varMatrix)))
    
    for iTime in range(1,len(currentWindow)):
        #minLog = min(logAlpha[:,iTime-1])
        #if math.isnan(minLog):
            #print("nan at time: ", iTime-1)
            #print("log alpha: ", logAlpha[:,iTime-1])
        for iState in range(0,nStates):
            alphaMat[iState,iTime] = sum(transitionMatrix[:,iState]*alphaMat[:,iTime-1])*normal(currentWindow[iTime],meanMatrix[0][iState],np.sqrt(varMatrix[0][iState]))
            if math.isnan(alphaMat[iState,iTime]):
                #print("nan alert: ",iTime)
                #print("initial prob: ",piVec)
                #print("mean vec: ",meanMatrix)
                print("var vec: ",varMatrix)
                #print(transitionMatrix[:,iState])
                #print(alphaMat[:,iTime-1])
                #print(normal(currentWindow[iTime],meanMatrix[0][iState],np.sqrt(varMatrix[0][iState])))
            alphaMatUnNorm[iState,iTime] = sum(transitionMatrix[:,iState]*alphaMatUnNorm[:,iTime-1])*normal(currentWindow[iTime],meanMatrix[0][iState],np.sqrt(varMatrix[0][iState]))
            #logAlpha[iState,iTime] = minLog + np.log(normal(currentWindow[iTime],meanMatrix[0][iState],np.sqrt(varMatrix[0][iState]))) + np.log(sum(transitionMatrix[:,iState]*np.exp(logAlpha[:,iTime-1]-minLog)))
        if sum(alphaMat[:,iTime]) > 0:
            alphaMat[:,iTime] = np.copy(alphaMat[:,iTime]/sum(alphaMat[:,iTime]))
        else:
            print('Divide by zero')
    
    #minLog = min(logAlpha[:,len(currentWindow)-1])
    #logPObs = np.log(sum(np.exp(logAlpha[:,len(currentWindow)-1]-minLog))) + minLog
    if sum(alphaMatUnNorm[:,len(currentWindow)-1]) == 0:
        print("Zero Final Prob")
    logPObs = np.log(sum(alphaMatUnNorm[:,len(currentWindow)-1]))
  
    return alphaMat, logPObs

In [194]:
def getBeta(currentWindow,nStates,piVec,transitionMatrix,meanMatrix,varMatrix):
    betaMat = np.ones((nStates,len(currentWindow)))
    betaMat[:,-1] = np.copy(betaMat[:,-1]/sum(betaMat[:,-1]))
    for iTime in range(2,len(currentWindow)+1):
        for iState in range(0,nStates):
            betaMat[iState,-iTime] = sum(betaMat[:,-iTime+1]*transitionMatrix[iState,:]*normal(currentWindow[-iTime+1],meanMatrix[0],np.sqrt(varMatrix[0])))
        betaMat[:,-iTime] = np.copy(betaMat[:,-iTime]/sum(betaMat[:,-iTime]))
    return betaMat

In [198]:
def tradeHMM(buyData, tradeStart, nStates, nComponents, piVec, transitionMatrix, meanMatrix, varMatrix, windowLength, threshold):
    prediction = np.zeros(len(buyData[0])-tradeStart)
    truth = np.zeros(len(buyData[0])-tradeStart)
    # true end = len(buyData[0])-1
    for uu in range(tradeStart,tradeStart+10):
        pObsNew = 10
        pObsCurrent = 0
        currentWindow = buyData[0,uu-windowLength:uu]
        pTracker = []
        while pObsNew - pObsCurrent > threshold:
                   
            alphaCurrent,pObsCurrent = getAlpha(currentWindow,nStates,piVec,transitionMatrix,meanMatrix,varMatrix)
            betaCurrent = getBeta(currentWindow,nStates,piVec,transitionMatrix,meanMatrix,varMatrix)
            pTracker.append(pObsCurrent)
            
            gammaCurrent = np.zeros((nStates,windowLength))
            for hh in range(0,windowLength):
                gammaCurrent[:,hh] = alphaCurrent[:,hh]*betaCurrent[:,hh]/sum(alphaCurrent[:,hh]*betaCurrent[:,hh])
                        
            newTransitionMatrix = np.zeros((nStates,nStates))
            originalGamma = np.zeros(nStates)
            for ii in range(0,nStates):
                originalGamma[ii] = sum(gammaCurrent[ii,0:windowLength-1])
            compareGamma = np.zeros(nStates)
            for ii in range(0,nStates):
                for jj in range(0,nStates):
                    for cc in range(0,windowLength-1):
                        currentDen = 0
                        for dd in range(0,nStates):
                            for ee in range(0,nStates):
                                currentDen = currentDen + alphaCurrent[dd,cc]*transitionMatrix[dd,ee]*normal(currentWindow[cc+1],meanMatrix[0][ee],np.sqrt(varMatrix[0][ee]))*betaCurrent[ee,cc+1]
                        newTransitionMatrix[ii,jj] = newTransitionMatrix[ii,jj] + (alphaCurrent[ii,cc]*transitionMatrix[ii,jj]*normal(currentWindow[cc+1],meanMatrix[0][jj],np.sqrt(varMatrix[0][jj]))*betaCurrent[jj,cc+1])/currentDen
                        if jj == 0:
                            for ff in range(0,nStates):
                                compareGamma[ii] = compareGamma[ii] + (alphaCurrent[ii,cc]*transitionMatrix[ii,ff]*normal(currentWindow[cc+1],meanMatrix[0][ff],np.sqrt(varMatrix[0][ff]))*betaCurrent[ff,cc+1])/currentDen
                    newTransitionMatrix[ii,jj] = newTransitionMatrix[ii,jj] / originalGamma[ii]
                    
            newPiVec = np.reshape(gammaCurrent[:,0],(nComponents,nStates))
            
            newMeanMatrix = np.zeros((nComponents,nStates))
            newVarMatrix = np.zeros((nComponents,nStates))
            #print("window ",currentWindow)
            for ii in range(0,nStates):
                valStore = np.copy(gammaCurrent[ii,:]*currentWindow)
                newMeanMatrix[0,ii] = sum(valStore)/sum(gammaCurrent[ii,:])
                valStore2 = np.copy(gammaCurrent[ii,:]*((currentWindow-newMeanMatrix[0,ii])**2))
                #print("mean store", ii, ": ",newMeanMatrix[0,ii])
                newVarMatrix[0,ii] = sum(valStore2)/sum(gammaCurrent[ii,:])
            #print(originalGamma)
            #print(compareGamma)
            #print("gamma: ",gammaCurrent)
            #print("new mean matrix: ",newMeanMatrix)
            print("new var mat: ",newVarMatrix)
            newAlpha,pObsNew = getAlpha(currentWindow,nStates,newPiVec,newTransitionMatrix,newMeanMatrix,newVarMatrix)
            pTracker.append(pObsNew)
            
            """
            if uu == tradeStart + 1:
                print("old transition",transitionMatrix)
                print("new transition", newTransitionMatrix)
                print("old pi", piVec)
                print("new pi", newPiVec)
                print("old mean", meanMatrix)
                print("new mean", newMeanMatrix)
                print("old var", varMatrix)
                print("new var", newVarMatrix)
                print("old alpha",alphaCurrent)
                print("new alpha",newAlpha)
            """
            piVec = np.copy(newPiVec)
            transitionMatrix = np.copy(newTransitionMatrix)
            meanMatrix = np.copy(newMeanMatrix)
            varMatrix = np.copy(newVarMatrix)
        
        viterbiPath = getViterbiPath(np.reshape(currentWindow,(1,len(currentWindow))),piVec,transitionMatrix, meanMatrix,varMatrix)
            
        prediction[uu] = np.sign(meanMatrix[0,np.argmax(transitionMatrix[viterbiPath[-1],:])]-currentWindow[-1])
        truth[uu] = np.sign(buyData[0,uu+1]-currentWindow[-1])
        print("p track", pTracker)
        print("pred",meanMatrix[0,np.argmax(transitionMatrix[viterbiPath[-1],:])])
        print("actual",buyData[0,uu+1])
        print("pred dir",prediction[uu])
        print("actual dir",truth[uu])
        print("next pi",piVec)
        print("next transition",transitionMatrix)
        print("next mean",meanMatrix)
        print("next var",varMatrix)
           
    return truth, prediction

In [200]:
buyData = np.reshape(buyPrices2_trade,(1,len(buyPrices2_trade)))
tradeStart = 10**4
truth,predictions = tradeHMM(buyData, tradeStart, 2, 1, piVec, transitionMatrix, meanMatrix, varMatrix, 100, 0.1)

new var mat:  [[ 54.81499675  25.04162249]]
new var mat:  [[ 42.23150531  25.0416225 ]]
p track [-416.82065873818192, -302.92082036897125, -302.92082036897125, -302.92082036444697]
pred 4770.0093
actual 4767.0
pred dir 1.0
actual dir 1.0
next pi [[  1.82482569e-08   9.99999982e-01]]
next transition [[  1.28566999e-04   9.99871433e-01]
 [  2.25481618e-10   1.00000000e+00]]
next mean [[ 4770.07770997  4770.0093    ]]
next var [[ 42.23150531  25.0416225 ]]
new var mat:  [[ 32.67659669  25.78012184]]
p track [-304.43963277329789, -304.37403950057632]
pred 4769.8604
actual 4767.0
pred dir 1.0
actual dir 1.0
next pi [[  1.72017465e-08   9.99999983e-01]]
next transition [[  1.35352245e-04   9.99864648e-01]
 [  2.26825012e-10   1.00000000e+00]]
next mean [[ 4771.06705117  4769.8604    ]]
next var [[ 32.67659669  25.78012184]]
new var mat:  [[ 19.31753664  25.60172404]]
p track [-304.0401450350551, -304.02683803341637]
pred 4769.7814
actual 4767.0
pred dir 1.0
actual dir 0.0
next pi [[  1.99710

C:\Users\draj36\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:13: RuntimeWarning: divide by zero encountered in log


In [139]:
-np.inf

-inf

In [38]:
backwardHours = 6
trade_cutoff = 2
zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync = formatTradeData(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes2_trade,sellPrices2_trade,buyTimes2_trade,buyPrices2_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slopeToUse_1,interceptToUse_1,slopeToUse_2,interceptToUse_2)

In [39]:
bankTracker1, naiveWorthTracker1, bankTracker2, naiveWorthTracker2, coinTracker1, coinTracker2, cashOutTimes,cashOut1, cashOut2, cashOutz1, cashOutz2 = pairs_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync,zScore1,zScore2,trade_cutoff)
plotReturn(timeLine,bankTracker1,naiveWorthTracker1,bankTracker2,naiveWorthTracker2,startTime,cashOutTimes,cashOut1+cashOut2,'huobi','okcoin')

In [46]:
fig, ax = plt.subplots()
ax.plot(timeLine,zScore1,'r',label=u'btce')
ax.plot(timeLine,zScore2,'b',label=u'bitstamp')
ax.plot(cashOutTimes,cashOutz1,'yo',label=u'btce cash out')
ax.plot(cashOutTimes,cashOutz2,'wo',label=u'bitstamp cash out')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
fig, ax = plt.subplots()
        labels.append(ax.get_xticklabels()[ii].get_text())
ax.plot(timeLine,zScore1,'r',label=u'btce')
ax.plot(timeLine,zScore2,'b',label=u'bitstamp')
ax.plot(cashOutTimes,cashOutz1,'yo',label=u'btce cash out')
ax.plot(cashOutTimes,cashOutz2,'wo',label=u'bitstamp cash out')   
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Z Score')
plt.title('Huobi and okcoin Z scores with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [47]:
fig, ax = plt.subplots()
ax.plot(timeLine,coinTracker1[1:],'r',label=u'btce')
ax.plot(timeLine,coinTracker2[1:],'b',label=u'bitstamp')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,coinTracker1[1:],'r',label=u'btce')
ax.plot(timeLine,coinTracker2[1:],'b',label=u'bitstamp') 
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Bitcoins')
plt.title('Huobi and Okcoin Bitcoins with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [53]:
fig, ax = plt.subplots()
ax.plot(timeLine,bankTracker1[1:],'r',label=u'btce')
ax.plot(timeLine,bankTracker2[1:],'b',label=u'bitstamp')
ax.plot(cashOutTimes,cashOut1,'yo',label=u'btce cash out')
ax.plot(cashOutTimes,cashOut2,'wo',label=u'bitstamp cash out')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,bankTracker1[1:],'r',label=u'btce')
ax.plot(timeLine,bankTracker2[1:],'b',label=u'bitstamp')
ax.plot(cashOutTimes,cashOut1,'yo',label=u'btce cash out')
ax.plot(cashOutTimes,cashOut2,'wo',label=u'bitstamp cash out')   
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Account Values')
plt.title('Huobi and OKcoin Account Values with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [54]:
timesInterest = timeLine[(cashOutTimes[8] < timeLine) & (timeLine < cashOutTimes[9] + 60)]
bank1 = bankTracker1[1:]
bank2 = bankTracker2[1:]
coins1 = coinTracker1[1:]
coins2 = coinTracker2[1:]
bank1Interest = bank1[(cashOutTimes[8] < timeLine) & (timeLine < cashOutTimes[9] + 60)]
bank2Interest = bank2[(cashOutTimes[8] < timeLine) & (timeLine < cashOutTimes[9] + 60)]
coin1Interest = coins1[(cashOutTimes[8] < timeLine) & (timeLine < cashOutTimes[9] + 60)]
coin2Interest = coins2[(cashOutTimes[8] < timeLine) & (timeLine < cashOutTimes[9] + 60)]

In [79]:
fig, ax = plt.subplots()
ax.plot(timesInterest,bank1Interest)
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timesInterest,bank1Interest)
ax.set_xticklabels(labels)
plt.title('Huobi account')
plt.show()

In [80]:
fig, ax = plt.subplots()
ax.plot(timesInterest,bank2Interest)
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timesInterest,bank2Interest)
ax.set_xticklabels(labels)
plt.title('OKcoin account')
plt.show()

In [81]:
fig, ax = plt.subplots()
ax.plot(timesInterest,coin1Interest)
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timesInterest,coin1Interest)
ax.set_xticklabels(labels)
plt.title('huobi coins')
plt.show()

In [82]:
fig, ax = plt.subplots()
ax.plot(timesInterest,coin2Interest)
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timesInterest,coin2Interest)
ax.set_xticklabels(labels)
plt.title('okcoin coins')
plt.show()

In [91]:
coin1Interest[-1]

0

In [84]:
bank2Interest[0]

11855.443767187322

In [85]:
bank1Interest[-1]

21441.728358006756

In [86]:
bank2Interest[-1]

4402.8212709548916

In [87]:
100*(((bank2Interest[-1]+bank1Interest[-1])/(bank1Interest[0]+bank2Interest[0]))-1)

18.265436932891998

In [33]:
import numpy as np
backwardHours = 6
trade_cutoff_vector = np.linspace(.6,4,40)
tradeReturn = np.zeros(len(trade_cutoff_vector))
zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync = formatTradeData(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes2_trade,sellPrices2_trade,buyTimes2_trade,buyPrices2_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slopeToUse_1,interceptToUse_1,slopeToUse_2,interceptToUse_2)
for ii in range(0,len(trade_cutoff_vector)): 
    trade_cutoff = trade_cutoff_vector[ii]
    bankTracker1, naiveWorthTracker1, bankTracker2, naiveWorthTracker2, coinTracker1, coinTracker2, cashOutTimes,cashOut1, cashOut2, cashOutz1, cashOutz2 = pairs_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync,zScore1,zScore2,trade_cutoff)
    tradeReturn[ii] = 100*(((bankTracker1[-1]+bankTracker2[-1])/20000)-1)
plt.plot(trade_cutoff_vector,tradeReturn,'b')
plt.xlabel('Z Score Threshold')
plt.ylabel('Percent Return')
plt.title('Pairs Trading Return')
plt.show()

In [34]:
import numpy as np
backwardHoursVec = [1/12,1/2,1,2,3,4,5,6,7,8,9,10,11,12]
trade_cutoff_vector = np.linspace(.6,4,20)
tradeReturn = np.zeros((14,len(trade_cutoff_vector)))
for ii in range(0,14):
    backwardHours = backwardHoursVec[ii]
    zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync = formatTradeData(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes2_trade,sellPrices2_trade,buyTimes2_trade,buyPrices2_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slopeToUse_1,interceptToUse_1,slopeToUse_2,interceptToUse_2)
    for jj in range(len(trade_cutoff_vector)):
        trade_cutoff = trade_cutoff_vector[jj]
        bankTracker1, naiveWorthTracker1, bankTracker2, naiveWorthTracker2, coinTracker1, coinTracker2, cashOutTimes,cashOut1, cashOut2, cashOutz1, cashOutz2 = pairs_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync,zScore1,zScore2,trade_cutoff)
        tradeReturn[ii,jj] = 100*(((bankTracker1[-1]+bankTracker2[-1])/20000)-1)

In [35]:
fig, ax = plt.subplots()
ax.imshow(tradeReturn)
xlabels = [item.get_text() for item in ax.get_xticklabels()]
ylabels = [item.get_text() for item in ax.get_yticklabels()]
ax.set_xticklabels(['','0.6','1.6','2.6','3.6',''])
ax.set_yticklabels(['','1/12','1','2','4','6','8','10','12',''])
plt.xlabel('Z-score threshold')
plt.ylabel('Backward Window (hours)')
plt.title('huobi and okcoin Pairs Trading Return')
plt.colorbar
plt.show()

In [153]:
aa = np.array([[1,2,3],[1,2,3]])
aa[:,3]

IndexError: index 3 is out of bounds for axis 1 with size 3